## Сравнительный анализ между PostgreSQL и MongoDB для нужд проекта CinemaUGC

### Зависимости для анализа
pymongo[srv]==4.6.1
psycopg2-binary==2.9.9
faker==22.0.0

In [1]:
from contextlib import contextmanager
import time
from random import choice

import psycopg2
from psycopg2.extras import execute_batch
import pymongo
import faker

### Подготовка
Создадим необходимые функции

In [2]:
@contextmanager
def postgres_cursor():
    connection = psycopg2.connect(password="admin", user="admin", dbname="postgres", host="localhost", port="5432")
    cursor = connection.cursor()
    try:
        yield cursor
        connection.commit()
    finally:
        cursor.close()
        connection.close()

@contextmanager
def mongo_client():
    client = pymongo.MongoClient(f"mongodb://localhost:27017/local")
    try:
        yield client.local
    finally:
        client.close()

Создадим первоначальную схему для таблиц PostgreSQL

In [113]:
with postgres_cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS likes, reviews, bookmarks")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS likes (
            movie_id UUID,
            user_id UUID,
            score SMALLINT
        );
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reviews (
            movie_id UUID,
            author_id UUID,
            text TEXT,
            has_spoiler BOOLEAN,
            attitude CHAR(8),
            created_at TIMESTAMP
        );
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS bookmarks (
            user_id UUID,
            movie_id UUID
        );
    """)

with mongo_client() as client:
    client.likes.drop()
    client.bookmarks.drop()
    client.reviews.drop()
    client.create_collection("likes")
    client.create_collection("bookmarks")
    client.create_collection("reviews")

Подготовим фейковые данные данные с помощью библиотеки faker

In [6]:
fake = faker.Faker()

USER_IDS = tuple(fake.uuid4() for _ in range(1000))
MOVIE_IDS = tuple(fake.uuid4() for _ in range(10000))
LIKE_AMOUNT = 200 * len(MOVIE_IDS)
REVIEW_AMOUNT = 5 * len(MOVIE_IDS)
BOOKMARK_AMOUNT = 25 * len(USER_IDS)

create_like = lambda: (choice(MOVIE_IDS), choice(USER_IDS), choice([0, 10]))
create_review = lambda: (choice(MOVIE_IDS), choice(USER_IDS), fake.text(), fake.boolean(), choice(["positive", "neutral", "negative"]), fake.date_time_this_year())
create_bookmark = lambda: (choice(USER_IDS), choice(MOVIE_IDS))

Заполним базы данных фейковыми данными

In [ ]:
with postgres_cursor() as cursor:
    execute_batch(cursor, "INSERT INTO likes (movie_id, user_id, score) VALUES (%s, %s, %s);", (create_like() for _ in range(LIKE_AMOUNT)))
    execute_batch(cursor, "INSERT INTO bookmarks (user_id, movie_id) VALUES (%s, %s);", (create_bookmark() for _ in range(BOOKMARK_AMOUNT)))
    execute_batch(cursor, "INSERT INTO reviews (movie_id, author_id, text, has_spoiler, attitude, created_at) VALUES (%s, %s, %s, %s, %s, %s);", (create_review() for _ in range(REVIEW_AMOUNT)))
    
with mongo_client() as db:
    db.likes.insert_many([{"movie_id": movie_id, "user_id": user_id, "score": score} for movie_id, user_id, score in (create_like() for _ in range(LIKE_AMOUNT))])
    db.bookmarks.insert_many([{"user_id": user_id, "movie_id": movie_id} for user_id, movie_id in (create_bookmark() for _ in range(BOOKMARK_AMOUNT))])
    db.reviews.insert_many([{"movie_id": movie_id, "author_id": author_id, "text": text, "has_spoiler": has_spoiler, "attitude": attitude, "created_at": created_at} for movie_id, author_id, text, has_spoiler, attitude, created_at in (create_review() for _ in range(REVIEW_AMOUNT))])

Покроем таблицы индексами

In [116]:
with postgres_cursor() as cursor:
    cursor.execute("CREATE INDEX IF NOT EXISTS likes_user_id_idx ON likes (user_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS likes_movie_id_idx ON likes (movie_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS reviews_author_id_idx ON reviews (author_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS reviews_movie_id_idx ON reviews (movie_id);")
    cursor.execute("CREATE INDEX IF NOT EXISTS bookmarks_user_id_idx ON bookmarks (user_id);")
    
with mongo_client() as db:
    db.likes.create_index([("movie_id", pymongo.ASCENDING), ("user_id", pymongo.ASCENDING)], unique=False)
    db.reviews.create_index([("movie_id", pymongo.ASCENDING), ("author_id", pymongo.ASCENDING)], unique=False)
    db.bookmarks.create_index([("user_id", pymongo.ASCENDING)], unique=False)

## Выполнение запросов к БД и сравнение скорости исполнения

---

### Поиск лайка для выбранного пользователя по выбранному фильму

In [10]:
# Get random likes from postgres
AMOUNT_OF_OPERATIONS = 1000

with postgres_cursor() as cursor:
    cursor.execute(f"SELECT COUNT(*) FROM likes")
    amount = cursor.fetchone()[0]

print(f"Size of table/amount of documents: {amount}")
print(f"Selected amount of operations: {AMOUNT_OF_OPERATIONS}")

# Get random likes from PostgreSQL
with postgres_cursor() as cursor:
    cursor.execute(f"SELECT user_id, movie_id FROM likes ORDER BY random() LIMIT {AMOUNT_OF_OPERATIONS}")
    likes = cursor.fetchall()

# PostgreSQL performance:
result = []
for user_id, movie_id in likes:

    start_time = time.monotonic()
    with postgres_cursor() as cursor:
        cursor.execute("SELECT * FROM likes WHERE user_id = %s AND movie_id = %s", (user_id, movie_id))
        like = cursor.fetchone()
    end_time = time.monotonic()

    result.append((end_time - start_time) * 1000)

print(f"Mean operation time for PostgreSQL: {sum(result) / len(result):.2f}ms")

# Get random likes from MongoDB
with mongo_client() as client:
    likes = list(client.likes.aggregate([{"$sample": {"size": AMOUNT_OF_OPERATIONS}}]))

# MongoDB performance:
result = []
for like in likes:
    user_id = like["user_id"]
    movie_id = like["movie_id"]

    start_time = time.monotonic()
    with mongo_client() as client:
        like = client.likes.find_one({"user_id": user_id, "movie_id": movie_id})
    end_time = time.monotonic()

    result.append((end_time - start_time) * 1000)

print(f"Mean operation time for MongoDB: {sum(result) / len(result):.2f}ms")

Size of table/amount of documents: 10002100
Selected amount of operations: 1000
Mean operation time for PostgreSQL: 10.65ms
Mean operation time for MongoDB: 7.26ms


Size of table/amount of documents: 10002100
Selected amount of operations: 1000
Mean operation time for PostgreSQL: 10.65ms
Mean operation time for MongoDB: 7.26ms

### Поиск всех закладок выбранного пользователя

In [4]:
AMOUNT_OF_OPERATIONS = 1000

with postgres_cursor() as cursor:
    cursor.execute(f"SELECT COUNT(*) FROM bookmarks")
    amount = cursor.fetchone()[0]

print(f"Size of table/amount of documents: {amount}")
print(f"Selected amount of operations: {AMOUNT_OF_OPERATIONS}")

# Get random user_ids
with postgres_cursor() as cursor:
    cursor.execute(f"SELECT user_id DISTINCT FROM bookmarks ORDER BY random() LIMIT {AMOUNT_OF_OPERATIONS}")
    user_ids = cursor.fetchall()

# PostgreSQL performance:
result = []
for user_id in user_ids:

    start_time = time.monotonic()
    with postgres_cursor() as cursor:
        cursor.execute("SELECT movie_id FROM bookmarks WHERE user_id = %s", user_id)
        cursor.fetchall()
    end_time = time.monotonic()

    result.append((end_time - start_time) * 1000)

print(f"Mean operation time for PostgreSQL: {sum(result) / len(result):.2f}ms")

# MongoDB performance:
result = []
for user_id in user_ids:

    start_time = time.monotonic()
    with mongo_client() as client:
        bookmarks = list(client.bookmarks.find({"user_id": user_id}))
    end_time = time.monotonic()

    result.append((end_time - start_time) * 1000)
print(f"Mean operation time for MongoDB: {sum(result) / len(result):.2f}ms")

Size of table/amount of documents: 125100
Selected amount of operations: 1000
Mean operation time for PostgreSQL: 8.74ms
Mean operation time for MongoDB: 6.19ms


Size of table/amount of documents: 125100
Selected amount of operations: 1000
Mean operation time for PostgreSQL: 8.74ms
Mean operation time for MongoDB: 6.19ms

### Создание новых записей в таблицах по методу один за другими

In [8]:
AMOUNT_OF_OPERATIONS = 1000
print(f"Selected amount of insert operations: {AMOUNT_OF_OPERATIONS}")

# Postgres performance
start_time = time.monotonic()
with postgres_cursor() as cursor:
    for _ in range(AMOUNT_OF_OPERATIONS // 10):
        cursor.execute("INSERT INTO bookmarks (user_id, movie_id) VALUES (%s, %s);", create_bookmark())
    for _ in range(int(AMOUNT_OF_OPERATIONS * 7 // 10)):
        cursor.execute("INSERT INTO likes (movie_id, user_id, score) VALUES (%s, %s, %s);", create_like())
    for _ in range(int(AMOUNT_OF_OPERATIONS * 2 // 10)):
        cursor.execute("INSERT INTO reviews (movie_id, author_id, text, has_spoiler, attitude, created_at) VALUES (%s, %s, %s, %s, %s, %s);", create_review())
end_time = time.monotonic()
print(f"PostgreSQL performance: {(end_time - start_time)*1000:.2f}ms")

# MongoDB performance
start_time = time.monotonic()
with mongo_client() as client:
    for _ in range(AMOUNT_OF_OPERATIONS // 10):
        client.bookmarks.insert_one(dict(zip(["user_id", "movie_id"], create_bookmark())))
    for _ in range(int(AMOUNT_OF_OPERATIONS * 7 // 10)):
        client.likes.insert_one(dict(zip(["movie_id", "user_id", "score"], create_like())))
    for _ in range(int(AMOUNT_OF_OPERATIONS * 2 // 10)):
        client.reviews.insert_one(dict(zip(["movie_id", "author_id", "text", "has_spoiler", "attitude", "created_at"], create_review())))
end_time = time.monotonic()
print(f"MongoDB performance: {(end_time - start_time)*1000:.2f}ms")

Selected amount of insert operations: 1000
PostgreSQL performance: 1462.89ms
MongoDB performance: 1068.52ms


Selected amount of insert operations: 1000
PostgreSQL performance: 1462.89ms
MongoDB performance: 1068.52ms

### Получение десяти фильмов с самой высокой оценкой

In [9]:
with postgres_cursor() as cursor:
    cursor.execute(f"SELECT COUNT(*) FROM likes")
    amount = cursor.fetchone()[0]

print(f"Size of table/amount of documents: {amount}")

# postgres performance:
start_time = time.monotonic()
with postgres_cursor() as cursor:
    cursor.execute("SELECT movie_id, AVG(score) as avg_score FROM likes GROUP BY movie_id ORDER BY avg_score DESC LIMIT 10")
    result = cursor.fetchall()
end_time = time.monotonic()
print(f"Mean operation time for PostgreSQL: {(end_time - start_time) * 1000:.2f}ms")


# mongo performance:
start_time = time.monotonic()
with mongo_client() as client:
    result = list(client.likes.aggregate([
        {"$group": {"_id": "$movie_id", "avg_score": {"$avg": "$score"}}},
        {"$sort": {"avg_score": -1}},
        {"$limit": 10}
    ]))
end_time = time.monotonic()
print(f"Mean operation time for MongoDB: {(end_time - start_time) * 1000:.2f}ms")

Size of table/amount of documents: 10002100
Mean operation time for PostgreSQL: 1127.10ms
Mean operation time for MongoDB: 6898.74ms


Size of table/amount of documents: 10002100
Mean operation time for PostgreSQL: 1127.10ms
Mean operation time for MongoDB: 6898.74ms

### Получение отзывов на 10 самых популярных фильмов (по количеству лайков/дислайков)

In [11]:
with postgres_cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM likes")
    likes_amount = cursor.fetchone()[0]
    cursor.execute("SELECT COUNT(*) FROM reviews")
    reviews_amount = cursor.fetchone()[0]
    
print(f"Amount of likes: {likes_amount}")
print(f"Amount of reviews: {reviews_amount}")

# postgres performance:
start_time = time.monotonic()
with postgres_cursor() as cursor:
    cursor.execute("""
        SELECT reviews.author_id, reviews.text
        FROM reviews
        JOIN (
            SELECT movie_id, COUNT(*) as likes_count
            FROM likes
            GROUP BY movie_id
            ORDER BY likes_count DESC
            LIMIT 10
        ) as top_movies ON top_movies.movie_id = reviews.movie_id
        WHERE reviews.has_spoiler = false AND reviews.attitude != 'negative';
    """)
    result = cursor.fetchall()
end_time = time.monotonic()
print(f"Mean operation time for PostgreSQL: {(end_time - start_time) * 1000:.2f}ms")

# mongo performance:
start_time = time.monotonic()
with mongo_client() as client:
    most_liked_movies = list(client.likes.aggregate([
        {"$group": {"_id": "$movie_id", "likes_count": {"$sum": 1}}},
        {"$sort": {"likes_count": -1}},
        {"$limit": 10}
    ]))
    result = list(client.reviews.find({
        "movie_id": {"$in": [movie["_id"] for movie in most_liked_movies]},
        "has_spoiler": False,
        "attitude": {"$ne": "negative"}
    }, {"author_id": 1, "text": 1}))
end_time = time.monotonic()
print(f"Mean operation time for MongoDB: {(end_time - start_time) * 1000:.2f}ms")

Amount of likes: 10002100
Amount of reviews: 250600
Mean operation time for PostgreSQL: 942.75ms
Mean operation time for MongoDB: 4552.17ms


Amount of likes: 10002100
Amount of reviews: 250600
Mean operation time for PostgreSQL: 942.75ms
Mean operation time for MongoDB: 4552.17ms

Итоги:
1. Поиск лайка для выбранного пользователя по выбранному фильму
- PostgreSQL: 10.65ms
- MongoDB: 7.26ms

2. Поиск всех закладок выбранного пользователя
- PostgreSQL: 8.59ms
- MongoDB: 6.38ms

3. Создание тысячи новых записей по методу один за другими без разрыва соединения
- PostgreSQL: 1462.89ms
- MongoDB: 1068.52ms

4. Получение десяти фильмов с самой высокой оценкой
- PostgreSQL: 1127.10ms
- MongoDB: 6898.74ms

5. Получение отзывов на 10 самых популярных фильмов (по количеству лайков/дислайков)
- PostgreSQL: 942.75ms
- MongoDB: 4552.17ms